In [1]:
from selenium import webdriver
import selenium
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
driver = webdriver.Chrome()
driver.set_page_load_timeout(15)
driver.get('https://sellercentral.amazon.com/forums/c/account-health')

for i in range(0,3):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

data = driver.find_elements_by_css_selector('a[class = "title raw-link raw-topic-link"]')
titles = [x.text for x in data]
links = [x.get_attribute('href') for x in data]

data_subcat = driver.find_elements_by_css_selector('span[class = "category-name"]')
sub_cats = [x.text for x in data_subcat[2:]]

contents = []
for link in links:
    driver.get(link)
    data = driver.find_elements_by_css_selector('div[class = "cooked"]')
    content_tempo = [x.text.replace('\n', ' ') for x in data]
    contents.append(content_tempo)

time.sleep(5)
driver.quit()

In [3]:
posts = []
responses = []
for line in contents:
    posts.append(line[0])
    responses.append(line[1:])

In [4]:
def create_response_df(urls, resps):
    column_names = ['URL', 'Response Text']
    df_resps = pd.DataFrame(columns = column_names)
    for url,line in zip(urls, resps):
        n = len(line)
        sub_urls = [url] * n
        new_df = pd.DataFrame(list(zip(sub_urls, line)), columns = column_names)
        df_resps = pd.concat([df_resps, new_df])
    return df_resps

In [5]:
df_main = pd.DataFrame(titles, columns=['Title'])
df_main['Category'] = 'Account Health'
df_main['Sub-category'] = sub_cats
df_main['Post Text'] = posts
df_main['URL'] = links

In [6]:
df_main.head()

,Title,Category,Sub-category,Post Text,URL
0,Unfounded Suspected IP Violations,Account Health,Intellectual Property,I had a listing removed by bot (not a complain...,https://sellercentral.amazon.com/forums/t/unfo...
1,3 Asin suspend - pls seller performance team a...,Account Health,Product Authenticity,Hi. It was the new account. I sent a total of ...,https://sellercentral.amazon.com/forums/t/3-as...
2,Amazon Covid 19 Bot Has Gotten Ridiculous,Account Health,Restricted Products,We have been selling dietary supplements on Am...,https://sellercentral.amazon.com/forums/t/amaz...
3,So this is how it is,Account Health,Account Health,I have been selling on Amazon for 3 years now....,https://sellercentral.amazon.com/forums/t/so-t...
4,Account Suspended After 5 years out of nowhere...,Account Health,Related Accounts,Our brand was suspended about 25 minutes ago. ...,https://sellercentral.amazon.com/forums/t/acco...


In [7]:
df_responses = create_response_df(links, responses)

In [8]:
df_responses.head()

,URL,Response Text
0,https://sellercentral.amazon.com/forums/t/unfo...,You need to get those thoughtful reps to refer...
1,https://sellercentral.amazon.com/forums/t/unfo...,Is there a specific department I should ask fo...
2,https://sellercentral.amazon.com/forums/t/unfo...,the_cosmetics You want to get somebody in cata...
0,https://sellercentral.amazon.com/forums/t/3-as...,"Hello @Gajols, Pax here from Amazon. Gajols: O..."
1,https://sellercentral.amazon.com/forums/t/3-as...,Hello Pax. Thank you for the answer. The last ...
